# Sequential Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies
Since the `OperatiotnsProblem` is the fundamental building block of a sequential
simulation in PowerSimulations, we will build on the [OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [2]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSimulations_examples", "1_operations_problems.jl"))

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/s


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalStandardUnitCommitment
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
  ser

### 5-Minute system
We had already created a `sys::System` from hourly RTS data in the OperationsProblem example.
The RTS data also includes 5-minute resolution time series data. So, we can create another
`System`:

In [3]:
sys_RT = System(rawsys; forecast_resolution = Dates.Minute(5))

┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:212
┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:212
┌ Info: Adding contributing generators for Spin_Up_R1 by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:652
┌ Info: Adding contributing generators for Spin_Up_R2 by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:652
┌ Info: Adding contributing generators for Spin_Up_R3 by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:652
┌ Info: Adding contributing generators for Flex_Up by category
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:652
┌ Info: Adding contributing gen

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
3,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
6,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,19
7,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,105
8,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,21
9,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51


## `OperationsProblemTemplate`s define `Stage`s
Sequential simulations in PowerSimulations are created by defining `OperationsProblems`
that represent `Stages`, and how information flows between executions of a `Stage` and
between different `Stage`s.

Let's start by defining a two stage simulation that might look like a typical day-Ahead
and real-time electricity market clearing process.

### Define the reference model for the day-ahead unit commitment

In [4]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_uc = template_unit_commitment(devices = devices)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalStandardUnitCommitment
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device_type = HVDCLine
        formulation = HVDCDispatch
  services: 
      ReserveDo

### Define the reference model for the real-time economic dispatch

In [5]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalDispatch
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device_type = HVDCLine
        formulation = HVDCDispatch
  services: 


In [6]:
using PowerGraphics
plotlyjs()

┌ Info: Precompiling PowerGraphics [5f7eddb3-86b1-49e8-a95a-ddc0f45eea41]
└ @ Base loading.jl:1260
[ Info: PowerGraphics.jl loads Plots.jl. Precompile might take a while


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16105742592585178449\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-11610665910635029045\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16105742592585178449\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

env: node: No such file or directory


Plots.PlotlyJSBackend()

### Define the `Stage`s
Stages define models. The actual problem will change as the stage gets updated to represent
different time periods, but the formulations applied to the components is constant within
a stage. In this case, we want to define two stages with the `OperationsProblemTemplate`s
and the `System`s that we've already created.

In [7]:
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template_uc, sys, solver),
    "ED" => Stage(GenericOpProblem, template_ed, sys_RT, solver, balance_slack_variables = true),
)

Dict{String,Stage{GenericOpProblem}} with 2 entries:
  "ED" => Stage()…
  "UC" => Stage()…

Note that the "ED" stage has a `balance_slack_variables = true` argument. This adds slack
variables with a default penalty of 1e6 to the nodal energy balance constraint and helps
ensure feasibility with some performance impacts.

### `SimulationSequence`
Similar to an `OperationsProblemTemplate`, the `SimulationSequence` provides a template of
how to execute a sequential set of operations problems.

print_struct(SimulationSequence)

Let's review some of the `SimulationSequence` arguments.

### Chronologies
In PowerSimulations, chronologies define where information is flowing. There are two types
of chronologies.
 - inter-stage chronologies: Define how information flows between stages. e.g. day-ahead
solutions are used to inform economic dispatch problems
 - intra-stage chronologies: Define how information flows between multiple executions of a
single stage. e.g. the dispatch setpoints of the first period of an economic dispatch problem
are constrained by the ramping limits from setpoints in the final period of the previous problem.

Let's define an inter-stage chronology that synchronizes information from 24 periods of
the first stage with a set of executions of the second stage:

In [8]:
feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

Dict{Pair{String,String},Synchronize} with 1 entry:
  "UC"=>"ED" => Synchronize(24)

### `FeedForward` and `Cache`
The definition of exactly what information is passed using the defined chronologies is
accomplished with `FeedForward` and `Cache` objects. Specifically, `FeedForward` is used
to define what to do with information being passed with an inter-stage chronology. Let's
define a `FeedForward` that affects the semi-continuous range constraints of thermal generators
in the economic dispatch problems based on the value of the unit-commitment variables.

In [9]:
feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_source_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

Dict{Tuple{String,Symbol,Symbol},SemiContinuousFF} with 1 entry:
  ("ED", :devices, :Generators) => SemiContinuousFF(:On, [:P], nothing)

### Sequencing
The stage problem length, look-ahead, and other details surrounding the temporal Sequencing
of stages are controlled using the `order`, `horizons`, and `intervals` arguments.
 - order::Dict(Int, String) : the hierarchical order of stages in the simulation
 - horizons::Dict(String, Int) : defines the number of time periods in each stage (problem length)
 - intervals::Dict(String, Dates.Period) : defines the interval with which stage problems
advance after each execution. e.g. day-ahead problems have an interval of 24-hours

So, to define a typical day-ahead - real-time sequence, we can define the following:
 - Day ahead problems should represent 48 hours, advancing 24 hours after each execution (24-hour look-ahead)
 - Real time problems should represent 1 hour (12 5-minute periods), advancing 1 hour after each execution (no look-ahead)

In [10]:
order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 24, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

Dict{String,Tuple{Hour,Consecutive}} with 2 entries:
  "ED" => (1 hour, Consecutive())
  "UC" => (24 hours, Consecutive())

Finally, we can put it all together:

In [11]:
DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
    )

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`
Now, we can build and execute a simulation using the `SimulationSequence` and `Stage`s
that we've defined.

In [12]:
sim = Simulation(
    name = "rts-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = rts_dir,
)

Simulation()


### Build simulation

In [13]:
build!(sim)

### Execute simulation

In [14]:
sim_results = execute!(sim)

Executing Step 1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: May 23 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.06949e+06 - 0.52 seconds
Cgl0003I 58 fixed, 0 tightened bounds, 1450 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 105 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1392 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1288 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1159 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 558 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 526 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 468 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 456 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tighten

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: May 23 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Presolve 180 (-3120) rows, 816 (-4212) columns and 984 (-7332) elements
Perturbing problem by 0.001% of 1000000 - largest nonzero change 0.10690343 ( 0.0082657072%) - largest zero change 0.092441989
0  Obj 5.8622404 Primal inf 101.98387 (172)
78  Obj 9991.0184 Primal inf 42.121187 (104)
156  Obj 15224.591 Primal inf 8.6740016 (31)
190  Obj 16247.747
Optimal - objective value 16241.026
After Postsolve, objective 16241.026, infeasibilities - dual 17044.996 (122), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 16241.026
Optimal - objective value 16241.026
Optimal objective 16241.0262 - 190 iterations time 0.012, Presolve 0.00
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: May 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: May 23 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Presolve 228 (-3072) rows, 1033 (-3995) columns and 1249 (-7067) elements
Perturbing problem by 0.001% of 1000000 - largest nonzero change 0.10783133 ( 0.0084223733%) - largest zero change 0.073501563
0  Obj 1343.7097 Primal inf 289.48669 (228)
79  Obj 18912.111 Primal inf 231.00982 (182)
158  Obj 37328.836 Primal inf 138.07866 (126)
237  Obj 44571.888 Primal inf 51.963286 (72)
316  Obj 49554.005 Primal inf 0.53278399 (2)
318  Obj 49554.905
Optimal - objective value 49539.608
After Postsolve, objective 49539.608, infeasibilities - dual 69205.589 (373), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 49539.608
Optimal - objective value 49539.608
Optimal objective 49539.60846 - 318 iterations time 0.012, Presolve 0.00
Total time (CPU seconds):       0.01   (Wal

Simulation Results Reference

Results Folder: /Users/cbarrows/Documents/repos/SIIPExamples.jl/RTS-GMLC-master/rts-test/5/results

Reference Tables

________________

stage-UC

P__ThermalStandard: dataframe size (1, 3)

P__RenewableDispatch: dataframe size (1, 3)

Reg_Down__VariableReserve_ReserveDown: dataframe size (1, 3)

parameter_P_PowerLoad: dataframe size (1, 3)

P__HydroEnergyReservoir: dataframe size (1, 3)

Flex_Down__VariableReserve_ReserveDown: dataframe size (1, 3)

parameter_service_requirement_VariableReserve_ReserveDown: dataframe size (1, 3)

Reg_Up__VariableReserve_ReserveUp: dataframe size (1, 3)

parameter_P_RenewableDispatch: dataframe size (1, 3)

Spin_Up_R2__VariableReserve_ReserveUp: dataframe size (1, 3)

parameter_service_requirement_VariableReserve_ReserveUp: dataframe size (1, 3)

parameter_P_HydroEnergyReservoir: dataframe size (1, 3)

parameter_P_RenewableFix: dataframe size (1, 3)

On__ThermalStandard: dataframe size (1, 3)

parameter_P_HydroDispatch: dataframe size (1, 3)

Flex_Up__VariableReserve_ReserveUp: dataframe size (1, 3)

Start__ThermalStandard: dataframe size (1, 3)

Stop__ThermalStandard: dataframe size (1, 3)

Spin_Up_R1__VariableReserve_ReserveUp: dataframe size (1, 3)

Spin_Up_R3__VariableReserve_ReserveUp: dataframe size (1, 3)

stage-ED

parameter_P_HydroDispatch: dataframe size (24, 3)

parameter_P_RenewableDispatch: dataframe size (24, 3)

P__ThermalStandard: dataframe size (24, 3)

P__RenewableDispatch: dataframe size (24, 3)

parameter_P_HydroEnergyReservoir: dataframe size (24, 3)

parameter_P_RenewableFix: dataframe size (24, 3)

parameter_P_PowerLoad: dataframe size (24, 3)

P__HydroEnergyReservoir: dataframe size (24, 3)

γ⁺__P: dataframe size (24, 3)

γ⁻__P: dataframe size (24, 3)

stage-UC

time length: 24

stage-ED

time length: 12

## Results

In [15]:
uc_results = load_simulation_results(sim_results, "UC");
ed_results = load_simulation_results(sim_results, "ED");

## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/PowerSimulations_examples/Plotting)

In [16]:
fuel_plot(uc_results, sys, load = true, curtailment = true)

data: [
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, title, and yaxis"

data: [
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y",
  "bar with fields barmode, marker, name, showlegend, stackgroup, type, x, and y"
]

layout: "layout with fields barmode, color, margin, title, and yaxis"

"Plots.PlotlyJSBackend() with 2 plots, named [:Fuel_Bar, :Fuel_Stack]"

In [18]:
plot_variable(uc_results, :P__ThermalStandard)

data: [
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, and y",
  "scatter with fields fillcolor, line, mode, name, showlegend, stackgroup, type, x, a

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*